In [6]:
import numpy as np
import pandas as pd
import time
import sys
import random
from helper_functions_assignment2 import*

In [7]:
#problem_instance = "inst_tuning\heur040_n_300_m_13358"
instance_type = "test_instances/" #"inst_competition/" #"inst_tuning/" # 
#problem_instance = "heur002_n_100_m_3274" #"heur002_n_100_m_3274" #"heur040_n_300_m_13358"
problem_instance = "heur001_n_10_m_31" 
path = "data/"+instance_type + problem_instance +".txt"

In [8]:
node_impact_orig, node_degree_orig, plex_assignment, edges_n1, edges_n2, edge_weights, edge_assignment_orig, s, n, m = create_problem_instance(path)
print(s)
print(n)
print(m)

1
10
31


In [9]:
plex_assignment = np.random.choice(3, n) #np.range(n) of size n (second is the size)
solution = assignment_to_solution(plex_assignment, node_impact_orig, 
                                    node_degree_orig, edge_weights, edge_assignment_orig, s)

old = solution.edge_assignment

print(np.where(old == 0))

plex_assignment, edge_assignment, removed = remove_random_node(plex_assignment, solution.edge_assignment, n, node_impact_orig, node_degree_orig, edge_weights)

with np.printoptions(threshold=np.inf):
    print(removed)
    print(np.where(edge_assignment == 0))

ValueError: kth(=1) out of bounds (1)

In [15]:
destroy_methods = [remove_random_node, remove_highest_cost_node, remove_most_edges_node, remove_smallest_splex, remove_largest_splex]
repair_methods = [add_to_smallest_splex, add_to_largest_splex, add_new_splex, add_to_random_splex]

destroy_weights = np.ones(len(destroy_methods))
repair_weights = np.ones(len(repair_methods))

# Parameters:
min_weight = 0.05
reaction_factor = 0.1
T_init = 10000
use_metropolis = True
cooling = 0.75
iterations_per_phase = 10
metropolis_success_value = 0.5 # Metropolis = partial success?

def update_weight(old_weight, num_applications, num_successes):
    if num_applications == 0:
        return old_weight
    new_weight = old_weight * (1-reaction_factor) + reaction_factor * (num_successes / num_applications)
    return max(new_weight, min_weight) # Make sure weight never equals 0


node_impact, node_degree, plex_assignment, edges_n1, edges_n2, edge_weights, edge_assignment_orig, s, n, m = create_problem_instance(path)
plex_assignment = np.random.choice(1, n) #np.range(n) of size n (second is the size)
solution = assignment_to_solution(plex_assignment, node_impact, 
                                    node_degree, edge_weights, edge_assignment_orig, s)

T = T_init
edge_assignment = solution.edge_assignment
current_score = sum(node_impact)/2

for i in range(100):
    destroy_prob = destroy_weights/destroy_weights.sum()
    repair_prob = repair_weights/repair_weights.sum()

    print(destroy_prob)
    print(repair_prob)

    destroy_applications = np.zeros(len(destroy_methods))
    destroy_successes = np.zeros(len(destroy_methods))
    repair_applications = np.zeros(len(repair_methods))
    repair_successes = np.zeros(len(repair_methods))

    for j in range(iterations_per_phase):
        destroy_idx = np.random.choice(len(destroy_methods), p=destroy_prob)
        repair_idx = np.random.choice(len(repair_methods), p=repair_prob)
        
        destroy_applications[destroy_idx] += 1
        repair_applications[repair_idx] += 1

        plex_assignment_new = plex_assignment.copy()
        edge_assignment_new = edge_assignment.copy()
        node_impact_new = node_impact.copy()
        node_degree_new = node_degree.copy()

        _, _, removed_nodes = destroy_methods[destroy_idx](plex_assignment_new, edge_assignment_new, n, node_impact_new, node_degree_new, edge_weights)
        repaired = repair_methods[repair_idx](plex_assignment_new, edge_assignment_new, n, removed_nodes, node_impact_new, node_degree_new, edge_weights, s)
        
        new_score = sum(node_impact_new)/2
        accept_solution = False
        if new_score < current_score:
            accept_solution = True
            destroy_successes[destroy_idx] += 1
            repair_successes[repair_idx] += 1
        elif use_metropolis:
            metropolis = math.exp(-abs(new_score-current_score)/T)
            P = random.uniform(0,1)
            if P < metropolis: # accept solution anyway
                accept_solution = True
                destroy_successes[destroy_idx] += metropolis_success_value
                repair_successes[repair_idx] += metropolis_success_value

        if accept_solution:
            current_score = new_score
            plex_assignment = plex_assignment_new
            edge_assignment = edge_assignment_new
            node_impact = node_impact_new
            node_degree = node_degree_new            

    for j in range(0, len(destroy_methods)):
        destroy_weights[j] = update_weight(destroy_weights[j], destroy_applications[j], destroy_successes[j])
        
    for j in range(0, len(repair_methods)):
        repair_weights[j] = update_weight(repair_weights[j], repair_applications[j], repair_successes[j])

    if use_metropolis:
        T *= cooling
    
    print("iter")

ValueError: kth(=3) out of bounds (3)

In [ ]:
get_edge_nodes(43, n)

(8, 10)